In [1]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_03/redo2/rois'

# 01 - Begin Analysis

Enter the path to the dictionary generated by Notebook 01

In [2]:
dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_01/rois/tmp/dataset_dict_pruned.json'

What correlation method would you like to use?
- Pearson is very fast, but depends on linearity and is sensitive to outliers
- Spearman is slower, but much more robust

In [3]:
correlation = 'spearman'

In [4]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
from calvin_utils.ccm_utils.stat_utils import CorrelationCalculator
data_loader = DataLoader(dict_path)
correlation_calculator = CorrelationCalculator(method=correlation, verbose=False)
corr_map_dict = correlation_calculator.generate_correlation_maps(data_loader)

Evaluating adni_Alzheimer
Evaluating corbetta_memory
Evaluating manitoba_memory
Evaluating ms_memory
Evaluating grafmann_memory
Evaluating maastricth_memory
Evaluating queensland_memory
Evaluating sante_memory
Evaluating kahana_random_stim
Evaluating wang_tms
Evaluating hebscher_tms
Evaluating hermiller_tms_a1
Evaluating hermiller_tms_b


Identify Datasets that Belong to Different Categories. 

Groups (values) allocate datasets to a fixed effect analysis. If no groups, set group_dict = {}
```
{
'dataset_1': 'group1',
'dataset_2': 'group2',
'dataset_3': 'group1'
}
```

In [5]:
corr_map_dict.keys()

dict_keys(['adni_Alzheimer', 'corbetta_memory', 'manitoba_memory', 'ms_memory', 'grafmann_memory', 'maastricth_memory', 'queensland_memory', 'sante_memory', 'kahana_random_stim', 'wang_tms', 'hebscher_tms', 'hermiller_tms_a1', 'hermiller_tms_b'])

Assign Dataset to Groups
- Datasets mapped to the same key will be merged into their own convergent map
- Leave as {} to merge all datasets into a single map

In [6]:
group_dict = {
    'sante_memory' : 'DBS',
    'queensland_memory' : 'DBS',
    'maastricth_memory' : 'DBS',
    'kahana_random_stim' : 'DBS',
    'adni_Alzheimer' : 'Lesion',
    'corbetta_memory' : 'Lesion',
    'ms_memory' : 'Lesion',
    'grafmann_memory' : 'Lesion',
    'wang_tms' : 'TMS',
    'hermiller_tms_a1' : 'TMS',
    'hermiller_tms_b' : 'TMS',
    'hebscher_tms' : 'TMS'
    }

# 02 - Use LOOCV to Train a Convergent Map on N-1 Datasets, Then Test it on the Nth dataset

Enter a list of datasets you want to flip (multiply correlation map by -1)
- This is typically done to control for sign inversion, enabling testing of topology specifically. 

In [7]:
datasets_to_flip = [] #'Alzheimer Disease', 'Multiple Sclerosis', 'Penetrating TBI'] #] #'fornix_memory', 'adni_Alzheimer', , 'grafmann_memory', 'manitoba_memory', 'ms_memory', 'kim_tms', 'hebscher_tms']

Generate the Convergent Maps and Save the Individual Correlation Maps

In [10]:
from calvin_utils.ccm_utils.convergent_loocv import LOOCVAnalyzer
loocv_analyzer = LOOCVAnalyzer(corr_map_dict, data_loader, 
                               similarity='spatial_correl',
                               method=correlation, 
                               n_bootstrap=1000, 
                               out_dir=out_dir,
                               weighting='unweighted',
                               mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii',
                               roi_path=None,
                               group_dict=group_dict,
                               datasets_to_flip=datasets_to_flip,
                               align_all_maps=False,
                               flip_axes=False
                               )

Removing NaNS from self.coor_map_dict.


Generate Agreement Maps

In [12]:
loocv_analyzer.generate_and_save_maps(subdir='convergence_maps/', verbose=True)

Generate Group Level Maps

In [13]:
loocv_analyzer.save_individual_r_maps(subdir='group_maps/', verbose=False)

Run LOOCV

In [ ]:
loocv_analyzer.run()

That's all

-Calvin